In [1]:
# load U-Net model and model weights 

from simple_unet_model import simple_unet_model 
from keras.utils import normalize
import os
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from skimage import measure, color, io


IMG_HEIGHT = 256
IMG_WIDTH  = 256
IMG_CHANNELS = 1

def get_model():
    return simple_unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

model = get_model()
model.load_weights('lamellipodia.hdf5')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 16  160         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 dropout (Dropout)              (None, 256, 256, 16  0           ['conv2d[0][0]']                 
                                )                                                             

In [75]:
# Conduct U-Net semantic segmentation

import imageio.v2 as imageio

def main():
    outPath = "C:/Users/lucyh/Desktop/Images/Raw images/Part 2/T30_Mock_0018_segmented/"
    os.makedirs(outPath)
    path = "C:/Users/lucyh/Desktop/Images/Raw images/Part 2/T30_Mock_0018/"

    # iterate through the images of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the image
        input_path = os.path.join(path, image_path)
        image_to_segment = cv2.imread(input_path, 0)
        
        # process the image
        image_to_segment = Image.fromarray(image_to_segment)
        image_to_segment = image_to_segment.resize((IMG_HEIGHT, IMG_WIDTH))
        test_img_norm = np.expand_dims(normalize(np.array(image_to_segment), axis=1),2)
        test_img_norm = test_img_norm[:,:,0][:,:,None]
        test_img_input = np.expand_dims(test_img_norm, 0)

        # segment 
        segmented = (model.predict(test_img_input)[0,:,:,0] > 0.5).astype(np.uint8)

        # save to folder 
        fullpath = os.path.join(outPath, 'segmented_'+image_path)
        imageio.imwrite(fullpath, segmented)

if __name__ == '__main__':
    main()

1/1 [==============================] - 0s 93ms/step


In [76]:
# Voronoi and otsu segmentation 

import sys
import cv2
from matplotlib import pyplot as plt
import pyclesperanto_prototype as cle
from skimage import io
import pandas as pd

def main():
    
    outPath = "C:/Users/lucyh/Desktop/Images/Raw images/Part 2/Measurements/"
    path = "C:/Users/lucyh/Desktop/Images/Raw images/Part 2/T30_Mock_0018_segmented/"
    
    data = []
    image = []

    # iterate through the images of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the image
        input_path = os.path.join(path, image_path)
        image_to_segment = io.imread(input_path)
        
        # step 1: heavy gaussian blurring to detect maxima (closer objects require decreased blurring)
        img_gaussian = cle.gaussian_blur(image_to_segment, sigma_x=3, sigma_y=3, sigma_z=3)
        img_maxima_locations = cle.detect_maxima_box(img_gaussian, radius_x=0, radius_y=0, radius_z=0)
        number_of_maxima_locations = cle.sum_of_all_pixels(img_maxima_locations)

        # step 2:threshold image after applying light gaussian blur
        img_gaussian2 = cle.gaussian_blur(image_to_segment, sigma_x=0.5, sigma_y=0.5, sigma_z=0.5)
        img_thresh = cle.threshold_otsu(img_gaussian2)
        
        # step 3: exclude maxima locations from the background
        img_relevant_maxima = cle.binary_and(img_thresh, img_maxima_locations)
        number_of_relevant_maxima_locations = cle.sum_of_all_pixels(img_relevant_maxima)

        # step 4: separate maximal locations into labels
        voronoi_separation = cle.masked_voronoi_labeling(img_relevant_maxima, img_thresh)
        
        # save count measurements
        image.append(image_path)
        data.append(number_of_relevant_maxima_locations)

    # export measurements
    df = pd.DataFrame(list(zip(image , data)), columns =['Image', 'Number of lamellipodia'])
    basename_without_ext = os.path.basename(os.path.dirname(path)) 
    df.to_excel(outPath + basename_without_ext + "_U-Net.xlsx", index=False)
    
if __name__ == '__main__':
    main()

In [83]:
# calculate average values

import pandas as pd

def main():
    
    outPath = "C:/Users/lucyh/Desktop/Images/Raw images/Part 2/Measurements/U-Net/"
    path = "C:/Users/lucyh/Desktop/Images/Raw images/Part 2/Measurements/U-Net/"
    
    folder = []
    x = []
    s = []
    
    # iterate through excel files of the folder
    for file in os.listdir(path):

        # create the full input path and read the image
        input_path = os.path.join(path, file)
        df = pd.read_excel(input_path)
        
        # calculate mean
        mean = df["Number of lamellipodia"].mean()
        se = df["Number of lamellipodia"].sem()
        
        # save values
        folder.append(file)
        x.append(mean)
        s.append(se)
    
    # export values
    df = pd.DataFrame(list(zip(folder, x, s)), columns = ['File', 'Average number of lamellipodia', 'Standard error'])
    df.to_excel(outPath + "U-Net_Compiled_Results.xlsx", index=False)
                    
if __name__ == '__main__':
    main()